# *Dataframe* operations
Before proceeding with the Water Flow exercise, let's first pause and discuss data structures: **vectors**/**series**, **matrices**, **N-dimensional arrays**, and particularly the **dataframe**.

## What is a dataframe?
The **dataframe** is a key tool in data analysis. Dataframes store data a specific format, one that facilitates many different types of analyses. This format is a table similar to an Excel worksheet, but has more strict conventions:
* Each **<u>column</u>** represents a **field**. It has a header or label and the values it holds all have the same **data type**.
* Each **<u>row</u>** represents an **observation**. All values in a given row related in that they describe the same entity. 
* Rows are typically referenced by an **<u>index</u>**. Index values are often, but not always, non-repeating sequential integer values.

## Typical operations in a dataframe
Once data are organized in a dataframe, it's quite straightforward to do the following:
* <u>Select</u>/<u>filter</u>/<u>sort</u> data by row, by column, or by both. 
* <u>Compute</u> new fields from existing ones
* <u>Combine</u> tables, either by appending columns or rows 
* <u>Reshape</u> tables either by melting, pivoting
* <u>Summarizing</u>/<u>Grouping</u> data
* Handling <u>missing data</u> 
* <u>Plotting</u>  data

We'll introduce how a few of these operations are done using the Python `Pandas` here in this notebook. Specifically, we'll examine how to subset rows and columns can be selected from dataframes as this offers more insight on how dataframes are organized and manipulated in Python. The other tasks will be examined in subsquent notebooks. 

## Diving in...
We'll begin by importing Pandas and then loading the water flow dataset retreived in the previous notebook. 

In [ ]:
#import libraries
import pandas as pd

In [ ]:
#Load data from the server into a dataframe named 'df'
url = 'http://waterservices.usgs.gov/nwis/dv/?format=rdb&sites=02087500&startDT=1930-10-01&endDT=2017-09-30&statCd=00003&parameterCd=00060&siteStatus=all'
df = pd.read_csv(url,
                 skiprows=31,
                 sep='\t',
                 names=['agency_cd','site_no','datetime','MeanFlow_cfs','Confidence'],
                 dtype={'site_no':'str'},
                 parse_dates=['datetime']
                )

In [ ]:
#Display each column's data type
df.dtypes

## Selecting data
Selecting data, aka "filtering", "subsetting", "slicing", etc., can be done by column, by row, or both. Identifying the rows or columns can be done by position, index/label, or by query, as we'll see in the following examples. 

### Selecting specific *columns* of data
Isolating a specific column of data is fairly straightforward; we just enter, in brackets the name of the column.

In [ ]:
#Create a new array from just the one column
dfMeanFlow = df['MeanFlow_cfs']

In [ ]:
#Show the first 5 rows of that array
dfMeanFlow.head()

To isolate more than one column, we just pass a *list* of column names between the brackets.<br>*Note that lists in Python are themselves surrounded by brackets `[]`.*

In [ ]:
#Create a new dataframe from the Flow and Confidence columns
df2 = df[['MeanFlow_cfs','Confidence']]
df2.head()

### Selecting rows of data...
There are a few means for selecting rows of data: by position, by index, or query, or by mask... Here, we'll touch on each.

### ♦ Selecting by position with `iloc`
First, we can pull one or a **slice** of rows by the row's sequential position in the dataframe using the `iloc` command (short for **i**nteger **loc**ation).<br>*Recall that in Python, lists begin at zero, not one...*

* Single values

In [ ]:
#Show first row
df.iloc[0]

In [ ]:
#Show second row
df.iloc[1]

In [ ]:
#Show last row
df.iloc[-1]

<font color=red>► What would the command be to show the 100th row of data?</font>

In [ ]:
#Show 100th row
df.iloc[]

* Data *slices*: A slice of data is a set of contiguous rows (or columns). We can slice our data with `iloc` by providing the bounds of the slice we want.<br>*Note that the upper bound is not included in the slice*. 

In [ ]:
#Show the first 4 rows. 
df.iloc[0:4]

In [ ]:
#Show the first 4 rows (again): Note that if we omit the lower bound, it assumes it's zero
df.iloc[:4]

In [ ]:
#Show rows 100 thru 105
df.iloc[99:104]

<font color=red>► What would the command be to show the last 5 records?</font>

In [ ]:
#Show the last 5 rows 
df.iloc[]

* Selecting rows *and columns* using `iloc`. 
Since tables are 2 dimensional, we can easily select/slice data by column or row AND column with `iloc.

In [ ]:
#First, remind us what our columns are
df.columns

In [ ]:
#Select the flow data (4th column) for the 100th record
df.iloc[99,3]

In [ ]:
#Select the flow data and confidence values for the 100th to 110th records
df.iloc[99:110,3:]

---
### ♦ Selecting by index with `loc`
While `iloc` references rows by their actual position in the data frame, `loc` references them by their **index**. Let's first examine this using the auto-generated indices created when we imported the CSV into a dataframe. Running the `index` function reveals that our initial index was assigned a sequential range of integers. 

In [ ]:
#What does our index look like? 
df.index

In [ ]:
#Show the rows corresponding to index values 6 thru 10
df.loc[6:10]

Now, let's change our index from the autogenerated sequential values to the values in stored in the `datetime` column.

In [ ]:
#Change the index to be values in the datetime column and display them
df.set_index('datetime',inplace=True)
df.index

In [ ]:
#Show the row with the index matching Jan 1st, 1975
df.loc['1975-01-01']

In [ ]:
#Show the slice of rows spanning september 10th thru 15th, 1998
df.loc['1998-09-10':'1998-09-15']

In [ ]:
#Return select rows AND columns using loc
df.loc['1998-09-10':'1998-09-15','MeanFlow_cfs':'Confidence']

<font color=red>► Use `loc` to return `MeanFlow_cfs` data for Sept 1, 2017</font>

In [ ]:
df.loc[]

<font color=red>► Use `loc` to return `MeanFlow_cfs` data Sept, 2017 onward to the end of the dataset</font>

In [ ]:
df.loc[]

### ♦ Selecting by querying data
Moving away from indices, we can query records matching criteria that we specify.  

In [ ]:
#Select rows where the Mean flow was less than 50 cfs
df.query('MeanFlow_cfs < 50')

In [ ]:
#Select rows where the Confidence indicates estimated:
df.query('Confidence == "A:e"')

<font color=red>► Query the data for mean flow values equal to 55</font>

In [ ]:
df.query()

## ♦ Using *masks* to query data
This method is a bit more convoluted. First we create a **mask** which is a binary column of data, meaning values are either true or false, by supplying a criteria. And then we **apply the mask**, which returns only those records that are true. 

In [ ]:
#Create a mask of flows below 53 cfs
maskTinyFlow = df['MeanFlow_cfs'] < 53

In [ ]:
#Apply the mask; this will only return rows where the mask was true
df[maskTinyFlow]

## Recap
Clearly, we are just scratching the surface of what we can do when our data is in a dataframe. However, in the next few notebooks, we'll dig a bit deeper by re-examining our water flow exercise. 